<a href="https://colab.research.google.com/github/casalazara/Data-Science-Aplicado/blob/master/Lab2/L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyreadstat

In [2]:
# Importante librerias o modulos que se van a usar en esta sesion
import os, sys
import numpy as np
import pandas as pd
import pandas_profiling 

##Librerías para graficar
import matplotlib
matplotlib.style.use('ggplot')
from matplotlib import pyplot as plt

# Lectura de los datos

In [3]:
condiVida = pd.read_spss('B.sav')
fuerTrabajo = pd.read_spss('H.sav')

In [4]:
condiVida.head()

,DIRECTORIO,FEX_C,REGION,DIA_REFERENCIA_2,clase,P424,P4030S1,P4030S1A1,P4030S5,P4030S3,P4030S4,P4030S4A1,P4030S2,P4020
0,2915591.0,463.993402,Central,Domingo,Cabecera,Casa,Sí,3.0,Sí,Sí,Sí,3.0,Sí,"Cemento, gravilla"
1,2915592.0,93.084381,Pacífica,Miércoles,Cabecera,Casa,Sí,1.0,No,No,Sí,3.0,No,"Baldosa, cerámica, laminado"
2,2915593.0,516.279638,Central,Domingo,Cabecera,Casa,Sí,1.0,Sí,Sí,Sí,3.0,Sí,"Baldosa, cerámica, laminado"
3,2915594.0,655.671443,Central,Domingo,Cabecera,Apartamento,Sí,1.0,Sí,Sí,Sí,3.0,Sí,"Cemento, gravilla"
4,2915595.0,326.783507,Central,Lunes,Cabecera,Apartamento,Sí,1.0,Sí,Sí,Sí,3.0,Sí,"Baldosa, cerámica, laminado"


In [5]:
fuerTrabajo.head()

,DIRECTORIO,SECUENCIA_P,ORDEN,P1152,P5953,P6260,P8340,P6280,P6290,P6300,P6310,P7310,P6330,P6340,P6350,P6351
0,2915591.0,1.0,1.0,Trabajando.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2915592.0,1.0,1.0,Trabajando.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2915593.0,1.0,1.0,Trabajando.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2915593.0,1.0,2.0,Estudiando.,No.,No.,No.,No.,NaN,No.,NaN,NaN,NaN,NaN,NaN,NaN
4,2915594.0,1.0,1.0,Trabajando.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# B. CONDICIONES DE LA VIVIENDA

## Análisis del tipo de variables

In [6]:
condiVida.dtypes

DIRECTORIO           float64
FEX_C                float64
REGION              category
DIA_REFERENCIA_2    category
clase               category
P424                category
P4030S1             category
P4030S1A1            float64
P4030S5             category
P4030S3             category
P4030S4             category
P4030S4A1            float64
P4030S2             category
P4020               category
dtype: object

De acuerdo a la descripción de variables en la página, las columnas P4030S1A1 y P4030S4A1 son las únicas variables numéricas discretas, las demás son continuas. Sin embargo, estas columnas cuentan con valores no numéricos que se deben reemplazar por lo que antes de cambiar el tipo hay que decidir qué hacer con ellos.

In [7]:
def cambiara_int(condiVida):
    condiVida["P4030S1A1"]=condiVida["P4030S1A1"].astype(int)
    condiVida["P4030S4A1"]=condiVida["P4030S4A1"].astype(int)

## Datos duplicados

In [8]:
duplicados = condiVida.duplicated().sum()
print(duplicados)
condiVida[condiVida.duplicated()]

0


,DIRECTORIO,FEX_C,REGION,DIA_REFERENCIA_2,clase,P424,P4030S1,P4030S1A1,P4030S5,P4030S3,P4030S4,P4030S4A1,P4030S2,P4020


## Datos faltantes

In [9]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 

In [10]:
missing_values_table(condiVida)

,Missing Values,% of Total Values
DIRECTORIO,0,0.000000
FEX_C,0,0.000000
REGION,0,0.000000
DIA_REFERENCIA_2,0,0.000000
clase,0,0.000000
P424,0,0.000000
P4030S1,0,0.000000
P4030S1A1,872,2.062197
P4030S5,0,0.000000
P4030S3,0,0.000000


La mayoría de columnas no tienen valores nulos. Sin embargo, las columnas P4030S1A1 y P4030S4A1 cuentan con valores nulos, la columna P4030S4A1 corresponde de acuerdo al diccionario a el número de días por semana que se realiza la recolección de basuras

In [11]:
(condiVida['P4030S4A1'].value_counts()*100/len(condiVida)).plot(kind='bar')


Mientras que la columna P4030S1A1 corresponde al estrato para la tarifa de la energía eléctrica

In [12]:
(condiVida['P4030S1A1'].value_counts()*100/len(condiVida)).plot(kind='bar')

Para la columna P4030S1A1 dada la baja cantidad de filas con este valor nulo y la importancia del estrato socioeconómico para los análisis lo mejor es eliminar estas filas, para la columna P4030S4A1 dada la distribución de valores lo mejor sería reemplazar los valores faltantes con la moda (3.0)

In [13]:
condiVida.dropna(axis=0, subset=['P4030S1A1'],inplace=True)
condiVida['P4030S4A1'] = condiVida['P4030S4A1'].fillna(condiVida['P4030S4A1'].mode()[0])
missing_values_table(condiVida)

,Missing Values,% of Total Values
DIRECTORIO,0,0.0
FEX_C,0,0.0
REGION,0,0.0
DIA_REFERENCIA_2,0,0.0
clase,0,0.0
P424,0,0.0
P4030S1,0,0.0
P4030S1A1,0,0.0
P4030S5,0,0.0
P4030S3,0,0.0


In [14]:
cambiara_int(condiVida)
condiVida.head()

,DIRECTORIO,FEX_C,REGION,DIA_REFERENCIA_2,clase,P424,P4030S1,P4030S1A1,P4030S5,P4030S3,P4030S4,P4030S4A1,P4030S2,P4020
0,2915591.0,463.993402,Central,Domingo,Cabecera,Casa,Sí,3,Sí,Sí,Sí,3,Sí,"Cemento, gravilla"
1,2915592.0,93.084381,Pacífica,Miércoles,Cabecera,Casa,Sí,1,No,No,Sí,3,No,"Baldosa, cerámica, laminado"
2,2915593.0,516.279638,Central,Domingo,Cabecera,Casa,Sí,1,Sí,Sí,Sí,3,Sí,"Baldosa, cerámica, laminado"
3,2915594.0,655.671443,Central,Domingo,Cabecera,Apartamento,Sí,1,Sí,Sí,Sí,3,Sí,"Cemento, gravilla"
4,2915595.0,326.783507,Central,Lunes,Cabecera,Apartamento,Sí,1,Sí,Sí,Sí,3,Sí,"Baldosa, cerámica, laminado"


## Análisis valores de las variables
Si una variable contiene valores no lógicos lo primero que haría sobre la base de datos sería mirar los datos de las filas con estos valores de manera que se pueda identificar si el error fue intencionado o no, si no lo fue tratar de mirar una forma de corregirlo de acuerdo al valor allí presente, en caso de no poderse dependría de la importancia de la variable para los análisis el desechar la fila o no.

In [15]:
col_interes = 'DIRECTORIO	REGION	clase	P424	P4030S1	P4030S1A1	P4030S5	P4030S3	P4030S4	P4030S4A1	P4030S2	P4020'.split("\t")
condiVida = condiVida[col_interes]
for columna in condiVida.columns:
  print(columna,"\n",condiVida[columna].unique())

DIRECTORIO 
 [2915591. 2915592. 2915593. ... 2965178. 2965179. 2965180.]
REGION 
 [Central, Pacífica, Oriental, Atlántica, San Andrés, Bogotá]
Categories (6, object): [Central, Pacífica, Oriental, Atlántica, San Andrés, Bogotá]
clase 
 [Cabecera, Resto]
Categories (2, object): [Cabecera, Resto]
P424 
 [Casa, Apartamento, Cuarto(s), Vivienda indígena, Otra vivienda (carpa, tienda, vagón, refugio n...]
Categories (5, object): [Casa, Apartamento, Cuarto(s), Vivienda indígena,
                         Otra vivienda (carpa, tienda, vagón, refugio n...]
P4030S1 
 [Sí]
Categories (1, object): [Sí]
P4030S1A1 
 [3 1 2 4 0 5 9 6]
P4030S5 
 [Sí, No]
Categories (2, object): [Sí, No]
P4030S3 
 [Sí, No]
Categories (2, object): [Sí, No]
P4030S4 
 [Sí, No]
Categories (2, object): [Sí, No]
P4030S4A1 
 [3 2 6 1 7 5 4 8]
P4030S2 
 [Sí, No]
Categories (2, object): [Sí, No]
P4020 
 [Cemento, gravilla, Baldosa, cerámica, laminado, Vinilo, tableta, ladrillo, madera pulida sin l..., Tierra, arena, Mármol, par

# H. FUERZA DE TRABAJO - Clasificacion

## Análisis del tipo de variables

In [16]:
fuerTrabajo.dtypes

DIRECTORIO      float64
SECUENCIA_P     float64
ORDEN           float64
P1152          category
P5953          category
P6260          category
P8340          category
P6280          category
P6290          category
P6300          category
P6310          category
P7310          category
P6330          category
P6340          category
P6350           float64
P6351          category
dtype: object

Las columnas P6350 y SECUENCIA_P de acuerdo a la información de las variables deberían ser discreta, las demás columnas están en el tipo que deberían estar. Sin embargo por la presencia de valores nulos en ellas no se puede hacer el cambio sin haberlos removido previamente

In [71]:
    def cambiar_int2(fuerTrabajo):
      #fuerTrabajo["P6350"]=fuerTrabajo["P6350"].astype(int) #Se remueve más adelante por la cantidad de ausencias
      fuerTrabajo["SECUENCIA_P"]=fuerTrabajo["SECUENCIA_P"].astype(int)

## Datos duplicados


In [18]:
duplicados = fuerTrabajo.duplicated().sum()
print(duplicados)
fuerTrabajo[fuerTrabajo.duplicated()]

0


,DIRECTORIO,SECUENCIA_P,ORDEN,P1152,P5953,P6260,P8340,P6280,P6290,P6300,P6310,P7310,P6330,P6340,P6350,P6351


## Datos faltantes

In [19]:
faltantes = missing_values_table(fuerTrabajo)
faltantes

,Missing Values,% of Total Values
DIRECTORIO,0,0.000000
SECUENCIA_P,0,0.000000
ORDEN,0,0.000000
P1152,0,0.000000
P5953,58706,47.606150
P6260,63366,51.385060
P8340,64905,52.633073
P6280,66042,53.555094
P6290,118465,96.066204
P6300,70893,57.488890


Hay una gran cantidad de columnas con más del 90% de valores faltantes por lo que es mejor removerlas y ver con más detalle la importancia de las que siguen presentando valores nulos


In [20]:
a_remover=[]
for columna in fuerTrabajo.columns:
  if faltantes.loc[columna,'% of Total Values'] >= 90:
    a_remover.append(columna)
fuerTrabajo.drop(axis=1,labels=a_remover,inplace=True)
missing_values_table(fuerTrabajo)

,Missing Values,% of Total Values
DIRECTORIO,0,0.000000
SECUENCIA_P,0,0.000000
ORDEN,0,0.000000
P1152,0,0.000000
P5953,58706,47.606150
P6260,63366,51.385060
P8340,64905,52.633073
P6280,66042,53.555094
P6300,70893,57.488890


Para las columnas con valores faltantes cercanos al 50% considero prudente eliminar las filas que tengan 2 o más de estas columnas vacías.



In [64]:
fuerTrabajo.dropna(subset=['P5953','P6260','P8340','P6280','P6300'],thresh=2,inplace=True)
missing_values_table(fuerTrabajo)

,Missing Values,% of Total Values
DIRECTORIO,0,0.000000
SECUENCIA_P,0,0.000000
ORDEN,0,0.000000
P1152,0,0.000000
P5953,0,0.000000
P6260,0,0.000000
P8340,1539,2.567139
P6280,2676,4.463720
P6300,7527,12.555463


In [66]:
for columna in (['P8340','P6280','P6300']):
  print(fuerTrabajo[columna].value_counts())

No.    57274
Sí.     1137
Name: P8340, dtype: int64
No.    52423
Sí.     4851
Name: P6280, dtype: int64
No.    44145
Sí.     8278
Name: P6300, dtype: int64


Hay una gran diferencia entre los dos valores de las columnas vacías por lo que considero apropiado reemplazar los valores faltantes con la moda.


In [67]:
for columna in (['P8340','P6280','P6300']):
  fuerTrabajo[columna] = fuerTrabajo[columna].fillna(fuerTrabajo[columna].mode()[0])


In [72]:
cambiar_int2(fuerTrabajo)
missing_values_table(fuerTrabajo)

,Missing Values,% of Total Values
DIRECTORIO,0,0.0
SECUENCIA_P,0,0.0
ORDEN,0,0.0
P1152,0,0.0
P5953,0,0.0
P6260,0,0.0
P8340,0,0.0
P6280,0,0.0
P6300,0,0.0


## Análisis valores de las variables

In [74]:
for columna in fuerTrabajo.columns:
  print(columna,"\n",fuerTrabajo[columna].unique())

DIRECTORIO 
 [2915593. 2915596. 2915597. ... 2965175. 2965179. 2965180.]
SECUENCIA_P 
 [1 2 3 4 5]
ORDEN 
 [ 2.  3.  4.  1.  5.  7.  8.  6. 10.  9. 11. 13. 16. 12. 14. 15. 17.]
P1152 
 [Estudiando., Oficios del hogar., Buscando trabajo., Otra actividad]
Categories (4, object): [Estudiando., Oficios del hogar., Buscando trabajo., Otra actividad]
P5953 
 [No.]
Categories (1, object): [No.]
P6260 
 [No., Sí.]
Categories (2, object): [No., Sí.]
P8340 
 [No., Sí.]
Categories (2, object): [No., Sí.]
P6280 
 [No., Sí.]
Categories (2, object): [No., Sí.]
P6300 
 [No., Sí.]
Categories (2, object): [No., Sí.]
